In [97]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, DateTime, Text, ForeignKey
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker

In [98]:
travel_sensor_df = pd.read_csv("travel_sensor.csv")
sensor_data_df = pd.read_csv("Radar_Traffic_Counts.csv")
traffic_detectors_df = pd.read_csv("traffic_detectors.csv")

In [99]:
traffic_detectors_df.head()

,Unnamed: 0,atd_location_id,created_date,detector_id,detector_status,detector_type,ip_comm_status,location_latitude,location_longitude,location_name,modified_date,signal_id
0,0,LOC16-004875,2017-06-28T05:00:00,1550,OK,LOOP,NaN,30.446960,-97.797707,9700 BLK ANDERSON MILL RD (Bethany),1498626000,997
1,1,LOC16-004875,2017-06-28T05:00:00,1551,OK,LOOP,NaN,30.446960,-97.797707,9700 BLK ANDERSON MILL RD (Bethany),1498626000,997
2,2,LOC16-004875,2017-06-28T05:00:00,1552,OK,LOOP,NaN,30.446960,-97.797707,9700 BLK ANDERSON MILL RD (Bethany),1498626000,997
3,3,LOC16-004875,2017-06-28T05:00:00,1553,OK,LOOP,NaN,30.446960,-97.797707,9700 BLK ANDERSON MILL RD (Bethany),1498626000,997
4,4,LOC16-004485,2017-06-26T05:00:00,1939,OK,LOOP,NaN,30.314329,-97.739349,1100 BLK W 45TH ST (TEXAS SCHOOL FOR THE BLIND),1498453200,910


In [100]:
sensor_data_df.head()

,index,Detector_ID,KITS_ID,Intersection_Name,Lane,Volume,Speed,Month,Day,Year,Hour,Minute,Day_of_Week,Direction
0,0,74,19,KINNEYLAMAR,Lane1,4,18,1,23,2018,22,15,2,None
1,1,92,24,LOOP 360LAKEWOOD,NB_out,103,40,12,16,2017,19,45,6,NB
2,2,42,14,Robert E LeeBarton Springs,SB,44,27,1,23,2018,21,45,2,SB
3,3,43,14,Robert E LeeBarton Springs,NB,13,28,1,23,2018,21,45,2,NB
4,4,87,23,BURNETPALM WAY,SB_out,26,41,1,23,2018,22,15,2,SB


In [101]:
travel_sensor_df.head()

,Unnamed: 0,atd_location_id,atd_sensor_id,coa_intersection_id,comm_status_datetime_utc,council_district,cross_st,cross_st_block,ip_comm_status,kits_id,...,modified_date,primary_st,primary_st_aka,primary_st_block,reader_id,sensor_mfg,sensor_status,sensor_type,signal_eng_area,turn_on_date
0,0,LOC17-010425,158,NaN,2018-03-29T03:35:00.000,9,,NaN,ONLINE,20.0,...,2017-07-03T11:04:00.000,CESAR CHAVEZ ST,NaN,700.0,NaN,Wavetronix,TURNED_ON,RADAR,CENTRAL,2015-10-15T00:00:00.000
1,1,LOC17-010430,159,5155709.0,2018-03-29T03:35:00.000,9,JOHANNA ST,101.0,ONLINE,17.0,...,2017-07-03T11:15:00.000,CONGRESS AVE,NaN,1900.0,NaN,Wavetronix,TURNED_ON,RADAR,SOUTHWEST,2016-04-04T00:00:00.000
2,2,LOC17-010435,160,5152101.0,2018-06-07T03:35:00.000,5,MANCHACA RD,2801.0,ONLINE,1.0,...,2017-07-03T11:43:00.000,LAMAR BLVD,NaN,2800.0,NaN,Wavetronix,TURNED_ON,RADAR,SOUTHWEST,2013-05-05T00:00:00.000
3,3,LOC16-001010,116,5150579.0,2018-03-30T03:35:00.000,"5,2",1ST ST,8901.0,ONLINE,NaN,...,NaN,SLAUGHTER LN,NaN,600.0,south_1st_slaughter,NaN,TURNED_ON,BLUETOOTH,SOUTHWEST,NaN
4,4,LOC16-000720,113,5154726.0,2018-03-29T03:35:00.000,"3,9",1ST ST,2301.0,ONLINE,NaN,...,NaN,OLTORF ST,NaN,600.0,south_1st_oltorf,NaN,TURNED_ON,BLUETOOTH,SOUTHWEST,NaN


In [102]:
engine = create_engine(f"sqlite:///austin_traffic2.sqlite")
conn = engine.connect()

In [103]:
class Detector(Base):
    __tablename__ = 'traffic_detector'
    __table_args__ = {'extend_existing': True} 
    index = Column(Integer(), primary_key=True)
    atd_location_id = Column(String(20))
    created_date = Column(String(25))
    detector_id = Column(Integer())
    detector_status = Column(String(20))
    detector_type = Column(String(20))
    ip_comm_status = Column(String(20))
    location_latitude = Column(Integer())
    location_longitude = Column(Integer())
    location_name = Column(String(50))
    modified_date = Column(String(25))
    signal_id = Column(Integer())

In [104]:
class Data(Base):
    __tablename__ = 'sensor_data'
    __table_args__ = {'extend_existing': True}
    index = Column(Integer(), primary_key=True)
    Detector_ID = Column(Integer())
    KITS_ID = Column(Integer())
    Intersection_Name = Column(String(30))
    Lane = Column(String(10))
    Volume = Column(Integer())
    Speed = Column(Integer())
    Month = Column(Integer())
    Day = Column(Integer())
    Year = Column(Integer())
    Hour = Column(Integer())
    Minute = Column(Integer())
    Day_of_Week = Column(Integer())
    Direction = Column(String(10))

In [105]:
class Sensor(Base):
    __tablename__ = 'travel_sensor'
    __table_args__ = {'extend_existing': True}
    index = Column(Integer(), primary_key=True)
    atd_location_id = Column(String(20))
    atd_sensor_id = Column(Integer())
    coa_intersection_id = Column(Integer())
    comm_status_datetime_utc = Column(String(50))
    council_district = Column(Integer())
    cross_st = Column(String(20))
    cross_st_block = Column(Integer())
    ip_comm_status = Column(String(20))
    kits_id = Column(Integer())
    landmark = Column(String(20))
    location = Column(String(70))
    location_latitude = Column(Integer())
    location_longitude = Column(Integer())
    location_name = Column(String(50))
    location_type = Column(String(20))
    modified_date = Column(String(25))
    primary_st = Column(String(20))
    primary_st_aka = Column(String(20))
    primary_st_block = Column(Integer())
    reader_id = Column(Integer())
    sensor_mfg = Column(String(20))
    sensor_status = Column(String(20))
    sensor_type = Column(String(20))
    turn_on_date = Column(String(25))

In [106]:
Base.metadata.create_all(engine)

In [107]:
travel_sensor_data = travel_sensor_df.to_dict(orient="records")
sensor_data_data = sensor_data_df.to_dict(orient="records")
traffic_detectors_data = traffic_detectors_df.to_dict(orient="records")

In [108]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = sqlalchemy.schema.MetaData(bind=engine)
metadata.reflect()

In [109]:
travel_sensor_table = sqlalchemy.Table('travel_sensor', metadata, autoload=True)

conn.execute(travel_sensor_table.delete())

conn.execute(travel_sensor_table.insert(), travel_sensor_data)

In [110]:
sensor_data_table = sqlalchemy.Table('sensor_data', metadata, autoload=True)

conn.execute(sensor_data_table.delete())

conn.execute(sensor_data_table.insert(), sensor_data_data)

In [111]:
traffic_detectors_table = sqlalchemy.Table('traffic_detector', metadata, autoload=True)

conn.execute(traffic_detectors_table.delete())

conn.execute(traffic_detectors_table.insert(), traffic_detectors_data)

In [112]:
engine.execute("select * from traffic_detector limit 2").fetchall()

[(1, 'LOC16-004875', '2017-06-28T05:00:00', 1550, 'OK', 'LOOP', None, 30.4469604, -97.7977066, '9700 BLK ANDERSON MILL RD (Bethany)', '1498626000', 997),
 (2, 'LOC16-004875', '2017-06-28T05:00:00', 1551, 'OK', 'LOOP', None, 30.4469604, -97.7977066, '9700 BLK ANDERSON MILL RD (Bethany)', '1498626000', 997)]

In [113]:
engine.execute("select * from sensor_data limit 2").fetchall()

[(0, 74, 19, 'KINNEYLAMAR', 'Lane1', 4, 18, 1, 23, 2018, 22, 15, 2, 'None'),
 (1, 92, 24, 'LOOP 360LAKEWOOD', 'NB_out', 103, 40, 12, 16, 2017, 19, 45, 6, 'NB')]

In [114]:
engine.execute("select * from travel_sensor limit 2").fetchall()

[(1, 'LOC17-010425', 158, None, '2018-03-29T03:35:00.000', 9, '  ', None, 'ONLINE', 20, None, "{'type': 'Point', 'coordinates': [-97.737262, 30.261476]}", 30.261476000000002, -97.737262, '700 BLK E CESAR CHAVEZ ST', 'ROADWAY', '2017-07-03T11:04:00.000', ' CESAR CHAVEZ ST', None, 700, None, 'Wavetronix', 'TURNED_ON', 'RADAR', '2015-10-15T00:00:00.000'),
 (2, 'LOC17-010430', 159, 5155709, '2018-03-29T03:35:00.000', 9, ' JOHANNA ST', 101, 'ONLINE', 17, 'Fulmore Middle School', "{'type': 'Point', 'coordinates': [-97.751737, 30.244513]}", 30.244513, -97.751737, ' CONGRESS AVE / JOHANNA ST (Fulmore Middle School)', 'ROADWAY', '2017-07-03T11:15:00.000', ' CONGRESS AVE', None, 1900, None, 'Wavetronix', 'TURNED_ON', 'RADAR', '2016-04-04T00:00:00.000')]